In [1]:
import os
import pandas as pd
import json
import folium
import numpy as np

In [2]:
import branca.colormap as cm

## Unemployment rate for active population in Europe at a country level

We start by creating a map of Europe with respect to the unemployment rate at a country level. The data we got
is from  [eurostat](http://ec.europa.eu/eurostat/data/database) and we use two datasets to be more accurate.
In order to create the map we extract relevant data from both dataframes and we merge the results in a clever way.

In [3]:
#For the European unemployment rate, we decide to take two data sets which are available at 
# 'http://ec.europa.eu/eurostat/web/products-datasets/-/lfsa_urgan' (for the 15-64 years old active population)
# and at 'http://ec.europa.eu/eurostat/tgm/table.do?tab=table&init=1&language=en&pcode=tsdec450&plugin=1 '
# for the active population both available from EuroStat 

#First dataframe:

#We read the data for a xls extension
data_lfsa = r'lfsa_urgan.xls'

#we create a dataframe by reading the previous file with a coherent header (the 10th row will be considered as the columns)
df_lfsa = pd.read_excel(data_lfsa,header = 10)
#we drop the 6th first rows because it contains information about the European Union and we only want the countries separatly
df_lfsa = df_lfsa.iloc[6:]
#We drop the lasts rows because it contains 'useless' information
df_lfsa = df_lfsa.iloc[:33]
#We rename the column to a coherent name
df_lfsa = df_lfsa.rename(columns={'GEO/TIME' : 'country'})
#Some countries doesnt have the good name, and some are useless (Former Yugoslav Republic of Macedonia) so we fix that:
df_lfsa.country = df_lfsa.country.replace('Germany (until 1990 former territory of the FRG)','Germany' )
df_lfsa = df_lfsa.drop(df_lfsa[df_lfsa.country == 'Former Yugoslav Republic of Macedonia, the'].index)
#We reset the index in order to be more readable
df_lfsa = df_lfsa.reset_index()
del df_lfsa['index']

#let's have a look at the head:
df_lfsa.head()

,country,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Belgium,7.5,7.0,7.9,8.3,7.2,7.6,8.4,8.5,8.5,7.8
1,Bulgaria,6.9,5.6,6.8,10.3,11.3,12.3,13.0,11.4,9.2,7.6
2,Czech Republic,5.3,4.4,6.7,7.3,6.7,7.0,7.0,6.1,5.1,4.0
3,Denmark,3.8,3.4,6.0,7.5,7.6,7.5,7.0,6.6,6.2,6.2
4,Germany,8.7,7.5,7.8,7.0,5.8,5.4,5.2,5.0,4.6,4.1


In [4]:
#we only want the most recent value so we extract the '2016' column
df_lfsa = df_lfsa[['country','2016']]
#let's have a look:
df_lfsa.head()

,country,2016
0,Belgium,7.8
1,Bulgaria,7.6
2,Czech Republic,4.0
3,Denmark,6.2
4,Germany,4.1


In [5]:
#Second dataframe:

#we read the data from the xls file
data_tsdec = r'Eurostat_Table_tsdec450FlagNoDesc_8697d85a.xls' # r'Eurostat_Table_tsdec450FlagNoDesc_8697d85a-4d2c-43e6-9324-5a4cc68ff86c.xls'
              
#we create the dataframe as before
df_tsdec = pd.read_excel(data_tsdec, header=3)
#we replace every ':' in the file by NaN
df_tsdec = df_tsdec.replace(':',np.NaN)
#we drop all the columns with 'unnamed'
cols = df_tsdec.columns
cols = [c for c in cols if 'Unnamed' in c]
df_tsdec = df_tsdec.drop(cols, axis=1)
#we rename the column 'geo\\time' by 'country in order to have the same name as before
df_tsdec = df_tsdec.rename(columns = {'geo\\time' : 'country'})
#we keep only the 36 first entries because the other are countries not in Europe (US and Japan) or useless data
df_tsdec = df_tsdec.iloc[:36]
#we drop the 4 first rows
df_tsdec = df_tsdec.iloc[4:]
#We reset the index in order to be more readable
df_tsdec = df_tsdec.reset_index()
del df_tsdec['index']

#let's have a look at the head
df_tsdec.head()

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Belgium,6.6,6.4,7.1,8.6,9.8,9.7,9.5,9.2,9.3,...,7.5,7.0,7.9,8.3,7.2,7.6,8.4,8.5,8.5,7.8
1,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.9,5.6,6.8,10.3,11.3,12.3,13.0,11.4,9.2,7.6
2,Czech Republic,NaN,NaN,NaN,4.3,4.3,4.0,3.9,4.8,6.5,...,5.3,4.4,6.7,7.3,6.7,7.0,7.0,6.1,5.1,4.0
3,Denmark,7.2,7.9,8.6,9.6,7.7,6.7,6.3,5.2,4.9,...,3.8,3.4,6.0,7.5,7.6,7.5,7.0,6.6,6.2,6.2
4,Germany,NaN,5.5,6.6,7.8,8.4,8.2,8.9,9.6,9.4,...,8.5,7.4,7.6,7.0,5.8,5.4,5.2,5.0,4.6,4.1


In [6]:
#we only want the most recent value so we extract the '2016' column
df_tsdec = df_tsdec[['country','2016']]

#Switzerland as missing value, to fix that we choose to use the data we found at 
#https://www.amstat.ch/seco/pressedoku/201701_Leporello_fr.pdf
df_tsdec.loc[df_tsdec.country == 'Switzerland','2016'] = 3.7

#let's have a look at the tail (we can see Switzerland):
df_tsdec.tail()

,country,2016
27,United Kingdom,4.8
28,Iceland,3.0
29,Norway,4.7
30,Switzerland,3.7
31,Turkey,10.9


Now that we have the two data frames, we can merge them to have a better idea of the unemployment rate of each countries in Europe, we decide to take the mean of the two to construct the final dataframe

In [7]:
#we compute the mean of the two dataframes
europe_unemployment_rate = (df_tsdec['2016']+df_lfsa['2016'])/2
#we construct the final dataframe
df_europe_unemployment = df_lfsa.copy()
df_europe_unemployment['2016'] = europe_unemployment_rate
df_europe_unemployment = df_europe_unemployment.rename(columns={'2016': 'unemployment rate (2016)'})

In [8]:
#Final data frame we will use
df_europe_unemployment.head()

,country,unemployment rate (2016)
0,Belgium,7.8
1,Bulgaria,7.6
2,Czech Republic,4.0
3,Denmark,6.2
4,Germany,4.1


>The last problem we are facing is that we do not have any data for some countries (like Belarus or Ukraine) and the topojson file provided to us got this countries to draw (even Israel is drawn) so we decided to rewrite the file keeping only the countries present in our dataframe 'df_europe_unemployment'

To get more freedom when plotting the map, we decided to convert the TopoJson file provided to a GeoJson file using the website : http://jeffpaine.github.io/geojson-topojson/.
This allows us to use the GeoJson folium method.

In [9]:
# First we load the json file
europe_country_geo = 'topojson/europe.geojson.json'
geo_europe_json_data = json.load(open(europe_country_geo))

#we make a list comprehension to choose only the countries present in our dataframe
geo_europe_json_data['features'] = [row for row in geo_europe_json_data['features']
                                    if row['properties']['NAME'] in df_europe_unemployment['country'].values]




We decided to take the ?? colormap which makes sens for what we want to represent:

In [10]:
# cm.linear.PiYG
# cm.linear.RdYlGn
# cm.linear.Spectral
# cm.linear.Dark2
cm.linear.Accent

In [11]:
# create custom color brewer
linear = cm.linear.Accent# cm.LinearColormap(['green','yellow','red'], vmin=3., vmax=10.)

In [12]:
# discretize the color scale using the quantiles of the distribution
linear.to_step(6,
               data=df_europe_unemployment['unemployment rate (2016)'],
               method='quantiles',
               round_method='int')

In [13]:
def mapTo0_1(number):
    """ Maps the unemployment percentage to the [0,1] interval. """
    max_ = np.max(df_europe_unemployment['unemployment rate (2016)'])
    min_ = np.min(df_europe_unemployment['unemployment rate (2016)'])
    return number / (max_ - min_)

In [14]:
# Create dictionary with countries as keys and unemployment as values
unemployment_dict = df_europe_unemployment.set_index('country')['unemployment rate (2016)']
# Check the brewer is working
linear(mapTo0_1(unemployment_dict['Greece']))

'#f0027f'

In [15]:
# https://stackoverflow.com/questions/34685652/choropleth-map-using-folium-and-pandas
# potential source of pb : not the same names in df and json

We finally got all the ingredients necesarry to construct the european map of the unemployment rate, we will now vizualise it using a choropleth map.

In [16]:
#We create the map
europe_map = folium.Map([54, 15], tiles='cartodbpositron', zoom_start=3.5)

#We loop over each country in the Json file, the loop is useful to add a specific popup to each country
for each in geo_europe_json_data['features']:
    #We create the country specific popup text using HTML
    popup = folium.Html('<b>' + str(each['properties']['NAME']) + '</b> <br>' + \
        'Unemployment Rate : ' + str(unemployment_dict[each['properties']['NAME']]) + '%'\
                        ,script=True)
    #We draw the country on the map
    folium.GeoJson(
        each,
        style_function=lambda feature: {
            'fillColor': linear(mapTo0_1(unemployment_dict[feature['properties']['NAME']])),
            'color' : 'black',
            'weight' : 2,
            'dashArray' : '5, 5'
            }
        ).add_to(europe_map).add_child(folium.Popup(popup))


#We then display the map
europe_map

>Comments :
The countries in the southern Europe have a higher unemployment rate. We see that Greece, Italy, Spain and Portugal are the most
impacted countries. The western Europe seems to be less touched by unemployment especially in the center and the north of Europe with countries like Switzerland, Germany or Norway.
We decided to split the intervals following the  quantiles $q = [0, 0.2, 0.45, 0.6, 0.8, 0.9]$.
Switzerland's unemployment rate is very low compared to the rest of the Europe, indeed only Iceland,Germany,Czech Republic,Norway and United Kingdom are at the same level.
All the countries in white are contries for which we do not have any data.

## Unemployment rate at the level of Swiss Cantons. 

We have to create an other Choroplet map, this time showing the unemployment rate at the level of swiss cantons. This time
we get the data from https://www.amstat.ch/v2/index.jsp.

In [17]:
#load the data for unemployment for each canton
data_swiss_canton = r'Taux_de_chômage_par_cantons.xlsx'


#creation of the dataframe by reading the excel file and setting all the columns, values..
df_swiss_canton = pd.read_excel(data_swiss_canton,header = 1)
df_swiss_canton = df_swiss_canton.reset_index()[['index', 'Taux de chômage']]
df_swiss_canton  = df_swiss_canton.rename(columns={"index": "canton_b", "Taux de chômage": "unemployment rate"})
df_swiss_canton = df_swiss_canton.iloc[:-1]

#look at the head
df_swiss_canton.head()

,canton_b,unemployment rate
0,Zurich,3.3
1,Berne,2.4
2,Lucerne,1.7
3,Uri,0.6
4,Schwyz,1.7


In [18]:
#As before, we converted the TopoJson file into a GeoJson file
ch_topo_json =  open('topojson/ch-cantons.geojson.json')
#load the json file
geo_swiss_json_data = json.load(ch_topo_json)

In [19]:
#The problem we occur now is that the name of the cantons we had from the xls file do not match with the name of the cantons
#of the geojson file, we fix this by taking the exact same name as the geojson has and we change our dataframe.


#Usefull list that contains all the canton's name as in the geojson file
canton = []
#creation of a dictionnary that links every canton to its'id 
canton_id = {}
#Every name is extracted from the file like this:
for i in geo_swiss_json_data['features']:
    name = i['properties']['name']
    canton.append(name)
    canton_id[name] = i['id']

#We manipulate the dataframe in order to fix the problem
df_swiss_canton['canton'] = canton
df_swiss_canton = df_swiss_canton[['canton','canton_b','unemployment rate']]

#look at the head
df_swiss_canton.head()

,canton,canton_b,unemployment rate
0,Zürich,Zurich,3.3
1,Bern/Berne,Berne,2.4
2,Luzern,Lucerne,1.7
3,Uri,Uri,0.6
4,Schwyz,Schwyz,1.7


In [20]:
# We create a dictionnary with the canton name and the unemployment rate
canton_unemployment_df = df_swiss_canton.set_index('canton')['unemployment rate']

In [21]:
#We create the switzerland map
swiss_coord = [46.91, 8.2]
swiss_map = folium.Map(swiss_coord, tiles='cartodbpositron', zoom_start=8)

#We loop over each canton to draw it and add a specific popup
for each in geo_swiss_json_data['features']:
    #We create a HTML popup text
    popup = folium.Html('<b>' + str(each['properties']['name']) + '</b> <br>' + \
        'Unemployment Rate : ' + str(canton_unemployment_df[each['properties']['name']]) + '%'\
                        ,script=True)
    #We then draw a canton
    folium.GeoJson(
        each,
        style_function=lambda feature: {
            'fillColor': linear(mapTo0_1(canton_unemployment_df[feature['properties']['name']])),
            'color' : 'black',
            'weight' : 2,
            'dashArray' : '5, 5'
            }
        ).add_to(swiss_map).add_child(folium.Popup(popup))

#Finally we vizualise the map
swiss_map

## Bonus

In order to divided the areas by the Röstigraben and in relation with https://fr.wikipedia.org/wiki/R%C3%B6stigraben , we decided to separate the cantons into 3 groups: the german, the french and the italian part of the Switzerland

In [22]:
#We first create 3 lists with the corresponding cantons
german = ['ZH','LU','UR','SZ','OW','NW','GL','ZG','SO','BS','BL','SH','AR','AI','SG','GR','AG','TG','BE']
french = ['VD','VS','NE','GE', 'JU','FR']
italian = ['TI']

In [23]:
#We uptdate our dataframe in order to have the id available
df_swiss_canton['id'] = list(canton_id.values())
#let's look at the head
df_swiss_canton.head()

,canton,canton_b,unemployment rate,id
0,Zürich,Zurich,3.3,ZH
1,Bern/Berne,Berne,2.4,BE
2,Luzern,Lucerne,1.7,LU
3,Uri,Uri,0.6,UR
4,Schwyz,Schwyz,1.7,SZ


Now that we have the id's we can properly calculate the unemployment rate's of each parts of the Switzerland according to the Röstigraben by doing the mean of all the cantons present in each part. (The rate is already normalized so we can just sum and divide by the total number of cantons present).

In [24]:
#We define 3 variables to help us calculating the unemployment rate of the 3 swiss'parts
german_rate,french_rate,italian_rate = 0,0,0

rate_part = []
#we loop all the element in our dataframe
for index, row in df_swiss_canton.iterrows():
    #if the id is german we update the german rate
    if(row['id'] in german):
        german_rate += row['unemployment rate']
    elif(row['id'] in french):
        french_rate += row['unemployment rate']
    else:
        italian_rate += row['unemployment rate']
        
german_rate = german_rate/len(german)
french_rate = french_rate/len(french)
italian_rate = italian_rate/len(italian)

Now that we have the rates for the three part of Switzerland, we will update our dataframe

In [25]:
rate_part = []
for index, row in df_swiss_canton.iterrows():
    if(row['id'] in german):
        rate_part.append(german_rate)
    elif(row['id'] in french):
        rate_part.append(french_rate)
    else:
        rate_part.append(italian_rate)
        
#actually update the dataframe
df_swiss_canton['unemployment rate by part'] = rate_part

#let's have a look at the 
df_swiss_canton

,canton,canton_b,unemployment rate,id,unemployment rate by part
0,Zürich,Zurich,3.3,ZH,2.000000
1,Bern/Berne,Berne,2.4,BE,2.000000
2,Luzern,Lucerne,1.7,LU,2.000000
3,Uri,Uri,0.6,UR,2.000000
4,Schwyz,Schwyz,1.7,SZ,2.000000
5,Obwalden,Obwald,0.7,OW,2.000000
6,Nidwalden,Nidwald,1.0,NW,2.000000
7,Glarus,Glaris,1.8,GL,2.000000
8,Zug,Zoug,2.3,ZG,2.000000
9,Fribourg,Fribourg,2.7,FR,4.083333


Now let's just create a map with unemployment rates between the areas divided by the Röstigraben

In [26]:
# Let us create the swiss map
swiss_coord = [46.91, 8.2]
swiss_map_roesti = folium.Map(swiss_coord, tiles='cartodbpositron', zoom_start=8)

#We create a dictionnary of the canton related to the unemployment rate by "language"
df_roesti_mean = df_swiss_canton.set_index('canton')['unemployment rate by part']

#We loop over each canton
for each in geo_swiss_json_data['features']:
    #We create a HTML popup for each canton with the rates
    popup = folium.Html('<b>' + str(each['properties']['name']) + '</b> <br>' + \
                'Unemployment Rate : ' + str(canton_unemployment_df[each['properties']['name']]) + '% <br>'\
                'Unemployment Rate Roesti : ' + str(df_roesti_mean[each['properties']['name']]) + '%' \
                , script=True)
    #We draw each canton
    folium.GeoJson(
        each,
        style_function=lambda feature: {
            'fillColor': linear(mapTo0_1(df_roesti_mean[feature['properties']['name']])),
            'color' : 'black',
            'weight' : 2,
            'dashArray' : '5, 5'
            }
        ).add_to(swiss_map_roesti).add_child(folium.Popup(popup))
    
#Finally we vizualise the map
swiss_map_roesti